In [15]:
#pip install torch

SyntaxError: invalid syntax (3011665414.py, line 1)

In [ ]:

#pip install pandas

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset

In [18]:
import torch
import torch.nn as nn

class MovieIDPredictor(nn.Module):
    def __init__(self, num_movie_ids, movie_id_embedding_dim=64, transformer_heads=8, transformer_layers=1, transformer_dim=64):
        super(MovieIDPredictor, self).__init__()
        # New movie_id embedding dimension
        self.movie_id_embedding_dim = movie_id_embedding_dim
        self.transformer_dim = transformer_dim

        # Ensure the embedding dimension for movie_id matches the transformer dimension
        self.movie_id_embedding = nn.Embedding(num_movie_ids, self.movie_id_embedding_dim)

        # TransformerEncoder expects the input dimension (d_model) to match the transformer_dim
        # Assuming transformer_dim is set to match movie_id_embedding_dim or vice versa
        self.transformer = nn.TransformerEncoder(nn.TransformerEncoderLayer(d_model=self.transformer_dim, nhead=transformer_heads), num_layers=transformer_layers)
        
        # Output layer to classify movie IDs (stays unchanged)
        self.fc_out = nn.Linear(self.transformer_dim, num_movie_ids)
        
    def forward(self, movie_id):
        movie_id_emb = self.movie_id_embedding(movie_id).view(-1, 1, self.movie_id_embedding_dim)
        
        x = self.transformer(movie_id_emb)
        x = x.view(-1, self.transformer_dim)  # Reshape for the output layer
        output = self.fc_out(x)
        return output


In [47]:
from torch.utils.data import Dataset, DataLoader
import torch
import os
import pandas as pd
import numpy as np

class MoviePairDataset(Dataset):
    def __init__(self, preferred_movie_ids, similar_movie_ids):
        """
        Args:
            preferred_movie_ids (list of int): The IDs of the preferred movies.
            similar_movie_ids (list of int): The IDs of movies similar to the preferred ones.
        """
        assert len(preferred_movie_ids) == len(similar_movie_ids), "The lists must have the same length."
        self.preferred_movie_ids = preferred_movie_ids
        self.similar_movie_ids = similar_movie_ids
        
    def __len__(self):
        return len(self.preferred_movie_ids)
        
    def __getitem__(self, idx):
        preferred_id = torch.tensor(self.preferred_movie_ids[idx], dtype=torch.long)
        similar_id = torch.tensor(self.similar_movie_ids[idx], dtype=torch.long)
        return preferred_id, similar_id


# path to the dataset folder
data_folder = "data/"

# list all CSV files in the data directory
csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

# load all CSV files into a single DataFrame
all_movies_df = pd.concat(
    (pd.read_csv(f"{data_folder}{file}") for file in csv_files),
    ignore_index=True
)[:1000]

print(all_movies_df)

# Adjust these thresholds based on your dataset and preferences
RATING_DIFF_THRESHOLD = 1.0  # Movies within this rating difference are considered similar
YEAR_DIFF_THRESHOLD = 5  # Movies within this range of years are considered similar

preferred_movie_ids = []
similar_movie_ids = []

for row_idx, row in all_movies_df.iterrows():
    # Find movies in the same genre
    same_genre_movies = all_movies_df[all_movies_df['genre'] == row['genre']]

    similar_movies = same_genre_movies[:6]

    # Further filter movies by rating and year proximity
    #similar_movies = same_genre_movies[
        #(np.abs(same_genre_movies['rating'] - row['rating']) <= RATING_DIFF_THRESHOLD) &
    #    (np.abs(same_genre_movies['year'] - row['year']) <= YEAR_DIFF_THRESHOLD)
    #]
    
    # Exclude the current movie from its similar list
    similar_movies = similar_movies[similar_movies['movie_id'] != row['movie_id']]
    
    # Append pairs to the lists
    for similar_row_idx, similar_row in similar_movies.iterrows():
        preferred_movie_ids.append(row_idx) #'movie_id'])
        similar_movie_ids.append(similar_row_idx) #['movie_id'])

# Deduplicate pairs while maintaining order
pairs = list(dict.fromkeys(zip(preferred_movie_ids, similar_movie_ids)))
preferred_movie_ids, similar_movie_ids = zip(*pairs)

# Convert back to lists if necessary
preferred_movie_ids = list(preferred_movie_ids)
similar_movie_ids = list(similar_movie_ids)

# Now you can create the dataset
dataset = MoviePairDataset(preferred_movie_ids=preferred_movie_ids, similar_movie_ids=similar_movie_ids)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

       movie_id             movie_name  year certificate  runtime  \
0     tt5433140                 Fast X  2023         NaN      NaN   
1    tt11564570            Glass Onion  2022       PG-13  139 min   
2    tt10365998          Infinity Pool  2023           R  117 min   
3    tt14138650      The Pale Blue Eye  2022           R  128 min   
4    tt15789492               Infiesto  2023       TV-MA   96 min   
..          ...                    ...   ...         ...      ...   
995   tt0831887             The Spirit  2008       PG-13  103 min   
996   tt4106514  The Postcard Killings  2020   Not Rated  104 min   
997  tt13751694                 Animal  2023         NaN  146 min   
998   tt1527788   The Man from Nowhere  2010           R  119 min   
999   tt5539054             You Get Me  2017       TV-MA   89 min   

                      genre  rating  \
0    Action, Crime, Mystery     NaN   
1      Comedy, Crime, Drama     7.2   
2    Crime, Horror, Mystery     6.5   
3    Crime, Hor

In [48]:
list(zip(preferred_movie_ids, similar_movie_ids))

[(0, 127),
 (0, 379),
 (0, 680),
 (0, 966),
 (1, 7),
 (1, 31),
 (1, 48),
 (1, 165),
 (1, 196),
 (2, 3),
 (2, 215),
 (2, 464),
 (2, 671),
 (2, 750),
 (3, 2),
 (3, 215),
 (3, 464),
 (3, 671),
 (3, 750),
 (4, 8),
 (4, 29),
 (4, 32),
 (4, 33),
 (4, 39),
 (5, 13),
 (5, 25),
 (5, 40),
 (5, 52),
 (5, 57),
 (6, 11),
 (6, 34),
 (6, 49),
 (6, 56),
 (6, 59),
 (7, 1),
 (7, 31),
 (7, 48),
 (7, 165),
 (7, 196),
 (8, 4),
 (8, 29),
 (8, 32),
 (8, 33),
 (8, 39),
 (9, 207),
 (9, 270),
 (9, 457),
 (9, 515),
 (9, 711),
 (10, 18),
 (10, 22),
 (10, 24),
 (10, 53),
 (10, 65),
 (11, 6),
 (11, 34),
 (11, 49),
 (11, 56),
 (11, 59),
 (12, 15),
 (12, 17),
 (12, 21),
 (12, 23),
 (12, 30),
 (13, 5),
 (13, 25),
 (13, 40),
 (13, 52),
 (13, 57),
 (14, 84),
 (14, 123),
 (14, 182),
 (14, 183),
 (14, 263),
 (15, 12),
 (15, 17),
 (15, 21),
 (15, 23),
 (15, 30),
 (16, 20),
 (16, 26),
 (16, 36),
 (16, 37),
 (16, 50),
 (17, 12),
 (17, 15),
 (17, 21),
 (17, 23),
 (17, 30),
 (18, 10),
 (18, 22),
 (18, 24),
 (18, 53),
 (18, 65)

In [54]:
import torch.nn as nn
import torch.optim as optim

# Assuming the MovieIDPredictor class has been modified as discussed previously
num_movie_ids = 10000  # Adjust according to the actual number of unique movie IDs in your full dataset
model = MovieIDPredictor(num_movie_ids=num_movie_ids, movie_id_embedding_dim=64, transformer_heads=8, transformer_layers=1, transformer_dim=64)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Assuming 'dataloader' is a DataLoader instance for the modified MoviePairDataset
num_epochs = 20
for epoch in range(num_epochs):
    for preferred_id, similar_id in dataloader:
        optimizer.zero_grad()
        outputs = model(preferred_id)
        loss = criterion(outputs, similar_id)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 1, Loss: 5.3477044105529785
Epoch 2, Loss: 4.022366046905518
Epoch 3, Loss: 4.446227073669434
Epoch 4, Loss: 4.734285354614258
Epoch 5, Loss: 4.442795753479004
Epoch 6, Loss: 4.235209941864014
Epoch 7, Loss: 4.5481181144714355
Epoch 8, Loss: 3.4555602073669434
Epoch 9, Loss: 3.080949306488037
Epoch 10, Loss: 3.0775864124298096
Epoch 11, Loss: 2.732221841812134
Epoch 12, Loss: 2.257632255554199
Epoch 13, Loss: 2.1706323623657227
Epoch 14, Loss: 1.9991681575775146
Epoch 15, Loss: 2.0261051654815674
Epoch 16, Loss: 2.415498971939087
Epoch 17, Loss: 2.0175204277038574
Epoch 18, Loss: 2.0319690704345703
Epoch 19, Loss: 2.101487159729004
Epoch 20, Loss: 1.7639541625976562


In [55]:
torch.save(model.state_dict(), 'movie_predictor_model.pth')

In [56]:
loaded_model = MovieIDPredictor(num_movie_ids=num_movie_ids)

# Load the state dictionary
loaded_model.load_state_dict(torch.load('movie_predictor_model.pth'))

# Set the model to evaluation mode
loaded_model.eval()

MovieIDPredictor(
  (movie_id_embedding): Embedding(10000, 64)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (fc_out): Linear(in_features=64, out_features=10000, bias=True)
)

In [57]:
correct = 0
total = 0
with torch.no_grad():  # No need to track gradients during evaluation
    for preferred_id, similar_id in dataloader:  # Assuming you have a dataloader for your evaluation dataset
        outputs = loaded_model(preferred_id)
        _, predicted = torch.max(outputs.data, 1)  # Get the index of the max log-probability
        total += similar_id.size(0)
        correct += (predicted == similar_id).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the evaluation dataset: {accuracy:.2f}%')


Accuracy of the model on the evaluation dataset: 17.82%


In [61]:
import torch

def predict(movie_id, n):
    """
    Predicts the top n recommended movie_ids for a given movie_id.

    Args:
    movie_id (int): The movie ID for which recommendations are to be made.
    n (int): The number of recommendations to return.

    Returns:
    list: A list of the top n recommended movie IDs.
    """
    # Convert movie_id to a tensor and add a batch dimension (batch size = 1)
    movie_id_tensor = torch.tensor([movie_id], dtype=torch.long)
    
    # Ensure the model is in evaluation mode
    loaded_model.eval()
    
    with torch.no_grad():  # Inference doesn't require gradient calculation
        # Get model output for the given movie_id
        outputs = loaded_model(movie_id_tensor)
        
        # Get the scores, ignore the first recommendation as it's the movie itself
        _, recommended_ids = torch.topk(outputs, n + 1, dim=1)
        
        # Convert to a list and remove the input movie_id from the recommendations
        recommended_ids = recommended_ids[0].tolist()
        if movie_id in recommended_ids:
            recommended_ids.remove(movie_id)
        else:  # If the movie_id is not in the top n+1, remove the last to keep n recommendations
            recommended_ids.pop()

    return recommended_ids[:n]

# Example usage
# Ensure your model and environment are properly set up before calling this function.
movie_id = 20  # Example movie ID
n = 5  # Number of recommendations
recommended_movie_ids = predict(movie_id, n)
print(f"Recommended Movie IDs for Movie ID {movie_id}: {recommended_movie_ids}")


Recommended Movie IDs for Movie ID 20: [36, 50, 37, 16, 26]
